In [1]:
import os
import sys
# Get the parent directory of the current working directory
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
# Add the parent directory to sys.path
sys.path.append(parent_dir)


In [2]:
import gdelt # for gdelt searchs
from tools.gkg_tools import * # for gkg searchs
# %run "../gkg_tools.py" # using magic command run to access the script from the parent directory

from nb_widgets import * # for widgets

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns
from datasets import Dataset


import nltk
from nltk.tokenize import word_tokenize, sent_tokenize


from transformers import pipeline, set_seed
import torch

# # GPU Timing (using GPU 1) else -1 for CPU
device_id = 1 if torch.cuda.is_available() else -1 

here


In [3]:
gkg = gkg_operator() # create a gkg operator
manga = pd.read_csv('..\\data\\manga_soup_labeled.csv') # We could alternatively use the subset of the queried data in the last section.
manga = manga[manga['sourcecommonname'].map(manga['sourcecommonname'].value_counts()) > 5]
# drop fandompost.com sourcecommonname
manga = manga[manga['sourcecommonname'] != 'fandompost.com']
# subset labels to 'ONE PIECE'
# manga = manga[manga['label'] == 'ONE PIECE']
gkg.get_gkg(data=manga.iloc[:,:].copy())
gkg.parse_urls() # parse urls in the gkg_query
gkg.gkg_query.head()


,gkgrecordid,date,sourcecollectionidentifier,sourcecommonname,documentidentifier,counts,v2counts,themes,v2themes,locations,...,one piece,dan da dan,dragon ball,star wars,naruto,hero academia,fruits basket,fairy tale,attack on titan,label
0,20241006010000-205,20241006010000,1,gamerant.com,https://gamerant.com/one-piece-oda-reveals-leg...,NaN,NaN,UNGP_FORESTS_RIVERS_OCEANS;IDEOLOGY;TAX_FNCACT...,"CRISISLEX_CRISISLEXREC,5955;IDEOLOGY,168;IDEOL...",NaN,...,0.978500,0.006184,0.001215,0.000770,0.003681,0.005521,0.000692,0.002582,0.000856,ONE PIECE
1,20241006014500-449,20241006014500,1,comicbook.com,https://comicbook.com/anime/news/dandadan-anim...,NaN,NaN,TAX_WORLDLANGUAGES;TAX_WORLDLANGUAGES_MANGA;CR...,"WB_652_ICT_APPLICATIONS,1543;WB_662_SOCIAL_MED...",NaN,...,0.023335,0.821875,0.022341,0.010567,0.052774,0.038077,0.008625,0.010975,0.011431,DAN DA DAN
2,20241006014500-506,20241006014500,1,comicbook.com,https://comicbook.com/anime/news/my-hero-acade...,NaN,NaN,TAX_FNCACT;TAX_FNCACT_HERO;UNGP_FORESTS_RIVERS...,"USPEC_POLICY1,1312;EPU_POLICY_SPENDING,1312;TA...",1#Japan#JA#JA#36#138#JA,...,0.020850,0.034011,0.010226,0.005078,0.019680,0.887402,0.008349,0.006942,0.007462,HERO ACADEMIA
3,20241006021500-259,20241006021500,1,screenrant.com,https://screenrant.com/spy-x-family-hints-jeev...,NaN,NaN,TAX_FNCACT;TAX_FNCACT_SPY;ARMEDCONFLICT;EPU_CA...,"TAX_FNCACT_FORGER,1445;TAX_FNCACT_FORGER,2011;...",NaN,...,0.298979,0.112409,0.062313,0.041012,0.137002,0.177780,0.045667,0.064593,0.060245,NO MATCH
4,20241006023000-421,20241006023000,1,gamerant.com,https://gamerant.com/best-dongua-watch/,"KILL#6##4#Xiao Yan, Shaanxi, China#CH#CH26#34....","KILL#6##4#Xiao Yan, Shaanxi, China#CH#CH26#34....",ARMEDCONFLICT;TAX_FNCACT;TAX_FNCACT_ARTISTS;ED...,"SOC_POINTSOFINTEREST_UNIVERSITY,8634;TAX_FNCAC...","4#Tang San, Fujian, China#CH#CH07#26.1049#118....",...,0.050670,0.318744,0.167921,0.030608,0.121249,0.155341,0.058683,0.051296,0.045490,NO MATCH


In [4]:
gkg.gkg_query['sourcecommonname'].value_counts()

sourcecommonname
screenrant.com          58
animenewsnetwork.com    57
gamerant.com            45
comicbook.com           25
bleedingcool.com         6
Name: count, dtype: int64

In [5]:
# Example field with semicolon-separated values
allnames_example = gkg.gkg_query['allnames'].iloc[0]

# Split the string by semicolons
split_parts = allnames_example.split(';')

# Group every 4th item
grouped_parts = [split_parts[i:i+3] for i in range(0, len(split_parts[:10]), 3)]

# Join the parts into a block format
block_format = '\n'.join(['; '.join(group) for group in grouped_parts])

# Print the block format
print(f"An example of a GKG Field to be tokenized for vectorization:\n[{block_format};...]")


An example of a GKG Field to be tokenized for vectorization:
[Sun God Nika,47; Sun God,133; Elbaf Arc,351
One Piece,700; Sun God,918; Sun God,2252
Sun God,2384; Sun God Nika,2453; Egghead Island Arc,2539
Sun God Nika,2617; Joy Boy,2649; One Piece,2697;...]


In [6]:
field_options = ['amounts', 'persons', 'v2persons', 'allnames', 'themes', 'v2tone', 'v2location', 'locations', 'v2organizations', 'organizations', 'v2counts', 'counts']
dropdown = create_dropdown_widget(field_options)

Dropdown(description='Select:', index=3, options=('amounts', 'persons', 'v2persons', 'allnames', 'themes', 'v2…

In [7]:

gkg.parse_gkg_field(dropdown.value)


,index,allnames_0,allnames_1
0,0,Sun God Nika,47
1,0,Sun God,133
2,0,Elbaf Arc,351
3,0,One Piece,700
4,0,Sun God,918
...,...,...,...
4697,190,One Piece,7244
4698,190,Shonen Jump,7394
4699,190,Manga Plus,7413
4700,190,One Piece,7471


In [8]:
gkg.tokenize_field(col_idx=1)

['Aaron Dismuke',
 'Abby Trott',
 'Abnormal Titans',
 'About One Piece',
 'Absolute Batman',
 'Absolute Dark',
 'Accursed Prince',
 'Acrobatic Silky',
 'Adam McArthur',
 'Adaptation Illustrated',
 'Address My Gripe',
 'Admiral Kizaru',
 'Adult Swim',
 'Advanced Armament',
 'Advanced Conqueror',
 'After Elbaf',
 'After-School Hanako-kun',
 'Aiko Yamagami',
 'Aira Shiratori',
 'Aizawa Shota',
 'Aka Akasaka',
 'Aka Arasaka',
 'Akane Kurokawa',
 'Akari Kit',
 'Akemi Okamura',
 'Aki Hamaji',
 'Aki Hamazi',
 'Akio Iyoku',
 'Akira Himekawa',
 'Akira Kurosawa',
 'Akira Toriyama',
 'Akita Publishing',
 'Akiyuki Tateyama',
 'Alejandro Saab',
 'Alex Louis Armstrong',
 'Alex Organ',
 'Alex Rider',
 'Alien RaceThath',
 'All For',
 'All For One',
 'All Might',
 'All This Time',
 'All-Powerful Brawler',
 'Allison Keith-Shipp',
 'Also Have Trouble Adjusting',
 'Alternative Rock',
 'Although Netflix',
 'Alto Yukimura',
 'Amanda Winn Lee',
 'Ami Fukushima',
 'Among Yokais',
 'Ancient Egypt',
 'Ancient W

In [9]:
gkg.field_tokens[:10]

['Aaron Dismuke',
 'Abby Trott',
 'Abnormal Titans',
 'About One Piece',
 'Absolute Batman',
 'Absolute Dark',
 'Accursed Prince',
 'Acrobatic Silky',
 'Adam McArthur',
 'Adaptation Illustrated']

In [10]:
gkg.vectorize_field()
gkg.vectorized_df

c:\Users\Corey Dearing\Desktop\gdelt\LLM_projects\tools\gkg_tools.py:141: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df = df.apply(pd.to_numeric, errors='ignore')


,aaron dismuke,abby trott,abnormal titans,about one piece,absolute batman,absolute dark,accursed prince,acrobatic silky,adam mcarthur,adaptation illustrated,...,zeno robinson,zeno robinson discuss battling all,zeno robinson reflect,zero key,zero vs royal guard,zeta reticuli,zhang chulan,zoan devil fruit,zodiac curse,zyuden sentai kyoryuger
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
187,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
188,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
189,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
gkg.vectorize_field(weight='weighted')
gkg.vectorized_df

c:\Users\Corey Dearing\Desktop\gdelt\LLM_projects\tools\gkg_tools.py:141: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df = df.apply(pd.to_numeric, errors='ignore')
c:\Users\Corey Dearing\Desktop\gdelt\LLM_projects\tools\gkg_tools.py:168: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  aligned_tokens = aligned_tokens.applymap(get_weight)  # Apply weighting function to each token count


,aaron dismuke,abby trott,abnormal titans,about one piece,absolute batman,absolute dark,accursed prince,acrobatic silky,adam mcarthur,adaptation illustrated,...,zeno robinson,zeno robinson discuss battling all,zeno robinson reflect,zero key,zero vs royal guard,zeta reticuli,zhang chulan,zoan devil fruit,zodiac curse,zyuden sentai kyoryuger
0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.693147,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
187,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
188,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
189,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0


In [12]:
gkg.get_fields_stats(weight='weighted')

    Top 10 Tokens by Non-Zero Percentage
                                     Non-Zero Percentage
one piece                                      17.277487
dragon ball                                    11.518325
akira toriyama                                 10.994764
shonen jump                                    10.471204
manga plus                                      9.947644
fragrant flower blooms with dignity             7.329843
author saka mikami                              7.329843
ball daima                                      7.329843
hero academia                                   7.329843
straw hat pirates                               6.806283

Top 10 Tokens by Mean Weighted Value (Non-Zero Only)
                 Mean Weighted Value (Non-Zero)
colette decides                        2.944439
yuji kaku                              2.708050
moon knight                            2.639057
creees lee                             2.564949
justina ireland                       

In [13]:
# top_tokens = ['one piece','sun god','elbaf arc','shonen jump','straw hat pirates','ifrit jambe', 'joy boy', 'east blue']
top_tokens = ['one piece','dragon ball', 'naruto','bleach','attack on titan','hero academia',
            'dandadan','dan da dan', 'fruits basket', 'jujutsu kaisen', "bizarre adventure", 'black clover', 'batman',]
# Find the intersection of columns in the dataframe and the list of top tokens
columns_to_keep = gkg.vectorized_df.columns.intersection(top_tokens)

# Subset the dataframe using the intersection
op_example = gkg.vectorized_df.loc[:, columns_to_keep].copy()
titles = gkg.gkg_query['title'].tolist()
print(titles)
# add titles to the dataframe as index
op_example.index = titles
# drop rows with all zeros
# op_example.drop(op_example[op_example.sum(axis=1) == 0].index, inplace=True)
op_example.drop(op_example[op_example.eq(0).all(axis=1)].index, inplace=True)
op_example

# Filter for rows with at least two non-zero column values
filtered_df = op_example[op_example.astype(bool).sum(axis=1) >= 2]

# Define a styling function for coloring
def color_larger_smaller(val, row):
    # Only consider numeric values (ignore non-numeric columns)
    try:
        val = float(val)  # Convert value to float for comparison
    except ValueError:
        return ''  # If the value is not numeric, do not apply any color
    
    numeric_row = row[pd.to_numeric(row, errors='coerce').notna()]  # Only keep numeric values
    
    # Filter non-zero values
    non_zero_values = numeric_row[numeric_row > 0]
    max_val = non_zero_values.max() if not non_zero_values.empty else 0
    top_non_zero_values = non_zero_values[non_zero_values == max_val]
    
    if len(top_non_zero_values) > 1:  # If top non-zero values are tied
        if val in top_non_zero_values.values:
            return 'color: red;'  # Tie values are red
    elif val == max_val:
        return 'color: lightblue;'  # Largest value is light blue
    elif val > 0:
        return 'color: yellow;'  # Other non-zero values are yellow
    return ''  # Zero values remain unstyled

# Apply styling row-wise
def style_larger_smaller(df):
    def style_row(row):
        return [color_larger_smaller(val, row) for val in row]
    return df.style.apply(style_row, axis=1)

# Add 'label' column to filtered dataframe with the column name of the largest value (in uppercase),
# or 'NO LABEL' if there is more than one maximum value
filtered_df['label'] = filtered_df.apply(
    lambda row: 'NO LABEL' if (row == row.max()).sum() > 1 else row.idxmax().upper(), axis=1
)

# Apply styling
styled_filtered_df = style_larger_smaller(filtered_df)

# Display in Jupyter Notebook
styled_filtered_df



['One Piece: Oda Reveals A Legendary God From Elbaf', 'Dandadan Premiere Earns Stamp of Approval From Hideo Kojima - ComicBook.com', 'My Hero Academia Season 7 Shares First Look at Finale - ComicBook.com', "Spy x Family Has Been Dropping Hints That A Key Character Is Related To Damian, But It's Easy To Miss", 'Best Dongua To Watch', 'Mission: Yozakura Family Anime Gets 2nd Season - News - Anime News Network', 'Vampire Knight: Memories Manga Ends in 11th Volume - News - Anime News Network', 'The 19-Panel Page In Absolute Batman #1 (Minor Spoilers)', '"Nah, I\'d Win": Jujutsu Kaisen\'s Finale Revealed the True Meaning of Gojo\'s Most Famous Words', 'NEWS: Vampire Knight: Memories Manga Ends in 11th Volume - Forum - Anime News Network', 'NEWS: Fairy Tail: 100 Years Quest Anime Casts Nana Mizuki - Forum - Anime News Network', 'NEWS: Oshi no Ko Anime Gets 3rd Season, Reveals More Cast [1/2] - Forum - Anime News Network', "Dragon Ball Super Was A Great Anime... But Its Return Will Be The Bes

C:\Users\Corey Dearing\AppData\Local\Temp\ipykernel_45680\2552646304.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['label'] = filtered_df.apply(


,bizarre adventure,dan da dan,dragon ball,fruits basket,hero academia,jujutsu kaisen,one piece,label
My Hero Academia Season 7 Really Saved the Anime - ComicBook.com,0.000000,0.000000,0.000000,0.000000,2.484907,0.693147,0.000000,HERO ACADEMIA
One Piece Anime Announces Historic Hiatus - ComicBook.com,0.000000,0.000000,0.693147,0.000000,0.000000,0.000000,1.609438,ONE PIECE
Jujutsu Kaisen's Cursed Energy Explained: Why All Sorcerers Are In Japan & Why It Makes The Series Revolutionary,0.693147,0.000000,0.693147,0.000000,0.000000,2.079442,0.000000,JUJUTSU KAISEN
Jujutsu Kaisen Doesn't Need a Sequel Anytime Soon,0.000000,0.000000,0.000000,0.000000,0.693147,2.079442,0.000000,JUJUTSU KAISEN
Dan Da Dan Officially Confirms It's Jujutsu Kaisen's True Successor By Revisiting The Hit Shonen's Big Theme,0.000000,2.079442,0.000000,0.000000,0.000000,1.386294,0.000000,DAN DA DAN
"10 Most Hateable Anime Characters Of All Time, Ranked",0.000000,0.000000,0.693147,0.000000,0.693147,0.000000,0.000000,NO LABEL
Who Are the World's Wealthiest Manga Creators?,0.000000,0.000000,0.693147,0.000000,0.000000,0.000000,0.693147,NO LABEL


In [14]:
gkg.calculate_token_stats(weight='weighted')

In [15]:
token_stats = gkg.token_stats
# Ensure that top_tokens is intersected with the token_stats index to avoid KeyError
top_token_stats = token_stats.loc[token_stats.index.intersection(top_tokens)]

top_token_stats

,Frequency,NonZeroPercentage,Variance,StdDev,MaxWeight,IDF,Skewness,Kurtosis,MeanWeightedNonZero,MeanWeightedAll
one piece,68.532573,17.277487,0.662875,0.814171,2.890372,1.725913,1.956471,2.137237,2.076745,0.358809
dragon ball,37.469151,11.518325,0.372835,0.610602,3.295837,2.116779,3.331530,10.609360,1.703143,0.196174
hero academia,25.438081,7.329843,0.256578,0.506535,2.772589,2.544223,3.849120,13.725352,1.817006,0.133184
jujutsu kaisen,16.591613,5.235602,0.156195,0.395215,2.639057,2.854378,4.704010,21.510366,1.659161,0.086867
dan da dan,13.107859,3.664921,0.129983,0.360532,2.397895,3.172832,5.259378,26.680325,1.872551,0.068628
bizarre adventure,6.907755,2.617801,0.058913,0.242721,2.302585,3.460514,7.376375,56.468428,1.381551,0.036166
fruits basket,5.257495,2.094241,0.043951,0.209645,2.079442,3.642836,8.428242,73.164785,1.314374,0.027526
